# Lab3.3 Training an emotion classifier using embeddings

Copyright: Vrije Universiteit Amsterdam, Faculty of Humanities, CLTL

### Table of Contents

* [Section 1: Quick introduction to embeddings](#section1)
* [Section 2: Loading the emotion data](#section2)
* [Section 3: Preparing the training and test data](#section3)
* [Section 4: Training and applying the model](#section4)
* [Section 5: Generating the test report](#section5)
* [Section 6: Applying the classifier to your own text](#section6)


## 1 Quick introduction to embeddings  <a class="anchor" id ="section1"></a> 

Extracting features manually can get us a long way. In addition to lemma and part-of-speech, people have used other information: features of the previous words (on the left) or the next words (on the right), whether the current word starts with a capital, whether it is an abbreviation, etc.

A recent alternative way to create a 'semantic' representation of a word is by word embeddings: mapping words (or phrases) from the vocabulary to vectors of real numbers. Conceptually it involves a mathematical embedding from a space with many dimensions per word to a continuous vector space with a much lower dimension. For this reason, they are called dense representations.

In linguistics, word embeddings were discussed in the research area of distributional semantics. The idea is to quantify and categorize semantic similarities between linguistic items based on their distributional properties in large samples of language data. The underlying notion is that "a word is characterized by the company it keeps" (Firth). Embeddings are however the weights in the hidden layer of a neural network that is trained to predict the contexts rather than representing the context in a vector directly.

### Reference:

For a nice explanation how word embedddings can improve classical bag-of-word approaches, check out this page:

https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html


In this section, we will load pre-trained word embeddings called word2vec, created by Google. The embeddings have 300 dimensions.

First, download the file from [Kaggle](https://www.kaggle.com/pkugoodspeed/nlpword2vecembeddingspretrained) or from [Google code archive](https://code.google.com/archive/p/word2vec/). Then, create a folder and unpack the word2vec file in that folder.

We will load the embedding model with the Gensim package that we used before.

In [1]:
from sklearn.metrics.pairwise import cosine_similarity
import gensim
import numpy as np

In [2]:
##### Change to path to the location of your local copy of the GoogleNews embeddings
##### It may take a  minute to load the model
path_to_model = '/Users/piek/Desktop/ONDERWIJS/data/word-embeddings/classical-models/GoogleNews-vectors-negative300.bin'
word_embedding_model = gensim.models.KeyedVectors.load_word2vec_format(path_to_model, binary=True)  



In [3]:
### this model has 300 dimensions so we set the number of features to 300

num_features = 300

In case you computer cannot handle big models such as the Google news model, you can also download one of the smaller 'Glove' datasets. These are provided as text files and 'gensim' provides a function to convert and load them. Note that these models have less domensions than the Google model and you need to adapt the number of features in the code below.

In [12]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

glove_file = datapath('/Users/piek/Desktop/ONDERWIJS/data/word-embeddings/classical-models/glove.6B.50d.txt')
tmp_file = get_tmpfile("test_word2vec.txt")

_ = glove2word2vec(glove_file, tmp_file)
word_embedding_model = KeyedVectors.load_word2vec_format(tmp_file)

### this model has 50 dimensions so we set the number of features to 50

num_features = 50

Instead of downloading the models to disk, 'gensim' also provides a downloader API to load the model from the web when needed. In the next cell, we use this API to download a word embeddding model trained on tweets. Note that these are GloVe embeddings built using Tweets as the name suggests. These vectors are based on 2B tweets, 27B tokens, 1.2M vocab, uncased. The original source can be found here: https://nlp.stanford.edu/projects/glove/. The 25 in the model name refers to the dimensionality of the vectors.

In [15]:
from gensim.models.word2vec import Word2Vec
import gensim.downloader as api
# download the model and return as object ready for use
word_embedding_model = api.load("glove-twitter-25")

### this model has 25 dimensions so we set the number of features to 25
num_features = 25

Let's check if the model works.

In [4]:
word1='cat'
word2='dog'
word1_vector=np.array(word_embedding_model[word1]).reshape(1, -1)
word2_vector=np.array(word_embedding_model[word2]).reshape(1, -1)
print(cosine_similarity(word1_vector, word2_vector))

[[0.7609458]]


## 2. Loading the emotion data  <a class="anchor" id ="section2"></a> 

In [5]:
import pandas as pd
filepath = './data/MELD/train_sent_emo.csv'
df = pd.read_csv(filepath)

# 3. Preparing the training and test data  <a class="anchor" id ="section3"></a> 

The following import are needed again:

In [6]:
import sklearn
import numpy as np
import nltk
from nltk.corpus import stopwords

In the previous notebook, we used CountVectorizer to obtain the full vocabulary of the data set and generate vectors for the one-hot-endcoing of each word. In these vectors, each slot represents a word and a value '1' indicates that the word was present in the utterance and a '0' means absence. This results is large and sparse vector representations for each utterance. We have also seen that we can weight the relevance of a word using the 'TF.IDF' function. This still results in large and sparse vectors but weights are more subtle. The down side is sparseness, lack of generalisation and lack robustness. 

In the following, we are going to represent the utterances by an embedding representation. In fact, we take the word embedding of each token in the utterance and add these together, after which we take the average. All the embeddings have the same number of dimensions in the same order. So if two tokens have a high weight for one dimension then their co-uccurrence in an utterance will enforce that weight. Note that by adding and taking the average, we normalize for the length of the utterance and the order of the tokens is not relevant.

We are going to define two customized function using 'def' to create an embedding representation for each utterance. These functions are taken from: https://www.kaggle.com/varun08/sentiment-analysis-using-word2vec

The first function, called 'featureVecMethod', takes the words of the utterance and the embedding model as parameters. The num_features parameter determines the size of the vector. 

In [16]:
unknown_words =[]
known_words = []
# Function to average all word vectors in a paragraph
def featureVecMethod(words, stopwords, model, modelword_index, num_features):
    # Pre-initialising empty numpy array for speed
    # This create a numpy array with the length of the num_features set to zero values
    featureVec = np.zeros(num_features,dtype="float32")
    nwords = 0
        
    for word in  words:
        if not word in stop_words: 
            if word in index2word_set:
                nwords = nwords + 1
                featureVec = np.add(featureVec,model[word])
                known_words.append(word)
            else:
                word = word.lower()
                if word in index2word_set:
                    nwords = nwords + 1
                    featureVec = np.add(featureVec,model[word])
                    #we keep track of the words detected
                    known_words.append(word)
                else:
                    #we keep track of the unknown words to see how well our model fits the data
                    unknown_words.append(word)
    # Dividing the result by number of words to get average
    featureVec = np.divide(featureVec, nwords)
    return featureVec

The next function just deals with all the data and creates the list of input vectors. This function calls the previous function

In [17]:
# Function for calculating the average feature vector
def getAvgFeatureVecs(texts, stopwords, model, modelword_index, num_features):
    counter = 0
    textFeatureVecs = np.zeros((len(texts),num_features),dtype="float32")
    for text in texts:
        # Printing a status message every 1000th text
        if counter%200 == 0:
            print("Review %d of %d"%(counter,len(texts)))
            
        textFeatureVecs[counter] = featureVecMethod(text, stopwords, model, modelword_index,num_features)
        counter = counter+1
    return textFeatureVecs

Now back to our input data. We iterate over the Pandas frame in the same way as before but now we extract for each utterance the embedding representation.

In [18]:
# Calculating average feature vector for training set
### This is the number of dimensions in the word2vec model used. 
###The Google news model has 300 dimensions but if you use a Glove model you may have to adapt this accordinlgy

#Converting Index2Word which is a list to a set for better speed in the execution.
#Allows for quicker lookup if the words exist
index2word_set = set(word_embedding_model.wv.index2word)
stop_words = set(stopwords.words('english'))

training_vectors = []
training_labels = []
for index, utterance in enumerate(df['Utterance']):
    ### Running this for all data requires a lot of memory and takes about an hour.
    ### For teaching purposes, it makes sense to limit the data
    ### we limit the data to the first 1000 utterances
    ##if index==2000:
    ##    break
    training_vectors.append(nltk.tokenize.word_tokenize(utterance))
    training_labels.append(df['Emotion'].iloc[index])

trainDataVecs = getAvgFeatureVecs(training_vectors, stop_words, word_embedding_model, index2word_set, num_features)
#### Due to the averaging, there could be infinitive values or NaN values. The next numpy function turns these value to "0" scores
trainDataVecs = np.nan_to_num(trainDataVecs)  

/Users/piek/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys


Review 0 of 9989
Review 200 of 9989
Review 400 of 9989
Review 600 of 9989
Review 800 of 9989
Review 1000 of 9989
Review 1200 of 9989
Review 1400 of 9989
Review 1600 of 9989
Review 1800 of 9989
Review 2000 of 9989
Review 2200 of 9989
Review 2400 of 9989
Review 2600 of 9989
Review 2800 of 9989
Review 3000 of 9989
Review 3200 of 9989
Review 3400 of 9989
Review 3600 of 9989
Review 3800 of 9989
Review 4000 of 9989
Review 4200 of 9989
Review 4400 of 9989
Review 4600 of 9989
Review 4800 of 9989
Review 5000 of 9989
Review 5200 of 9989
Review 5400 of 9989
Review 5600 of 9989
Review 5800 of 9989
Review 6000 of 9989
Review 6200 of 9989
Review 6400 of 9989
Review 6600 of 9989
Review 6800 of 9989
Review 7000 of 9989
Review 7200 of 9989
Review 7400 of 9989
Review 7600 of 9989
Review 7800 of 9989
Review 8000 of 9989
Review 8200 of 9989
Review 8400 of 9989
Review 8600 of 9989
Review 8800 of 9989
Review 9000 of 9989
Review 9200 of 9989
Review 9400 of 9989
Review 9600 of 9989
Review 9800 of 9989


/Users/piek/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in true_divide


Training the classifier may take a while. If you laptop cannot handle it, reduce the number of training data. Alternatively, you can use a smaller word2vec embeddings model. Here is a website with many ready to use models: http://vectors.nlpl.eu/repository/

You can either choose a model with a smaller vocabulary or with less dimensions. Whatever you choose, make sure you can load the model using the 'gensim' package. If you choose a model with less than 300 dimensions (e.g. 100 or 200), you also need to adapt the value for *num_features* accordingly = 300.

Let's inspect our training data a bit more. Depending on the break set for loading the training data, you will have a list of vectors with according length:

In [19]:
len(trainDataVecs)

9989

We can inspect the first element in the list:

In [20]:
print('Vector length', len(trainDataVecs[0]))
print (trainDataVecs[0])

Vector length 300
[ 8.44395980e-02 -3.91489677e-02 -3.98813896e-02  1.29191086e-01
 -7.57039413e-02  3.86454277e-02  4.34163399e-02 -1.05855308e-01
  7.66703263e-02 -1.61132812e-02 -2.07354221e-02 -2.13623047e-01
  4.65799980e-02 -1.03851318e-01 -1.14990234e-01  3.64176445e-02
  4.88281250e-02 -2.69266758e-02 -4.19108057e-03 -1.00046791e-01
  6.75455714e-03  6.46769181e-02  8.85009766e-02 -2.89204922e-02
  1.74560547e-02 -1.18052168e-02 -1.38753250e-01  2.27457676e-02
  2.07519531e-02 -9.46044922e-03 -4.80143242e-02 -1.43178301e-02
 -5.51554374e-02 -1.12501783e-02  1.15275063e-01  4.34875488e-03
  5.06896973e-02 -4.02221680e-02 -4.39453125e-02  4.84415703e-02
  9.26513672e-02 -2.76285801e-02  1.28906250e-01  1.61126461e-02
  6.06689453e-02 -1.81477871e-02 -7.12687150e-02  1.75679531e-02
 -6.51448593e-02  3.47493477e-02  6.72200546e-02 -3.51053886e-02
  1.53808594e-02 -6.93257675e-02  9.27480031e-03  7.63346329e-02
 -7.54394531e-02 -3.40169258e-02  3.14737968e-02 -9.62320939e-02
 -4.943

It is simply a list with digits, each representing the averaged weight of the tokens or words that made up the utterance. We can checks the length, which should be '300', '100', '50' or '25', etc. depending on the number of dimensions of the word2vec model that you used.

There are two major differences with the bag-of-tokens that we used in the previous notebook:

1. the vectors are short
2. there are no zero's 

Instead of *large sparse* vectors, we now have *short dense* vectors representing each utterance. Whereas in the previous representation, each slot in the vector corresponds with a token, now each slot is a weight from the hidden layer to learn to predict others words in the context.

This is true for each utterance, each having a unique set of values for the same hidden layer weights. These weights now represent the meaning of the utterance for a machine, which can use a similarity function such as cosine similairty to measure the degree of equivalence across these representations. When we inspects any other utterance, we see it is represented in a simlar way.

In [21]:
print(len(trainDataVecs[1000]))
print(trainDataVecs[1000])

300
[-0.01953125  0.1188151  -0.05525716  0.10058594 -0.07259115 -0.01420085
 -0.10164388  0.0633138   0.04850261  0.10223898  0.04638672 -0.12011719
 -0.0460612   0.07820638 -0.10758463  0.18945312  0.24088542  0.26106772
 -0.15364583 -0.08650716  0.03776042  0.15722656  0.2084961   0.07409668
  0.16845195  0.09130859 -0.20377605  0.08414713 -0.0386556   0.04886882
 -0.13769531  0.04764811  0.2063802  -0.1110026   0.10302734  0.11572266
 -0.06542969  0.09765625 -0.02050781  0.05655925  0.21777344 -0.07486979
  0.19905598  0.10103353  0.06445312  0.07438151 -0.02278646 -0.04264323
  0.01302083 -0.04980469  0.03352864  0.18147786  0.00911458 -0.04402669
  0.03971354  0.0616862  -0.04947917 -0.03222656  0.10253906 -0.13330078
 -0.08317057  0.15234375 -0.38085938 -0.12141927 -0.04394531  0.12109375
  0.02132162  0.01106771 -0.22672527 -0.0320638   0.06217448 -0.02473958
  0.07779948  0.04931641 -0.26497397 -0.01839193 -0.08691406  0.11287435
  0.07181803  0.20703125  0.02392578  0.0188395

Since the vectors are compatible, we can compare them in the same way as we did before for the word2vec embeddings of *cat* and *dog*:

In [22]:
word1_vector=np.array(trainDataVecs[0]).reshape(1, -1)
word2_vector=np.array(trainDataVecs[1000]).reshape(1, -1)
print(cosine_similarity(word1_vector, word2_vector))

[[0.33458045]]


For training, we use the same labels as before:

In [23]:
print(training_labels[0], training_labels[1])

neutral neutral


So now we have a numeric representation of each text, based on the embeddings of the words. We feed this to a classifier in the same way as we did in the previous notebooks with the Countvectorizer output.

Before we can train the classifier, we stil need to convert the labels to numeric values as we did before.

Before we do that, it may be good to check which words are not in the embedding model and therefore do not contribute to the representation of the utterance. In the above function, we kept track of the unknown words. Now we can inspect this list. We use the *Counter* function to get a frequency count of these words.

In [24]:
from collections import Counter

unknown_words_count = Counter(unknown_words)
print('Proportion of unknown tokens', len(unknown_words)/(len(unknown_words)+len(known_words)))
print('Number of unknown words',len(unknown_words_count))
print('Number of unknown word tokens:', len(unknown_words))
print('Unknown words counts')
print(unknown_words_count)

Proportion of unknown tokens 0.3712182372983402
Number of unknown words 1029
Number of unknown word tokens: 27129
Unknown words counts
Counter({',': 7237, '.': 6476, '!': 4266, '?': 2887, "'s": 661, 'i\x92m': 496, 'don\x92t': 372, 'it\x92s': 334, '...': 301, 'that\x92s': 248, 'you\x92re': 222, 'y\x92know': 157, 'can\x92t': 132, 'we\x92re': 98, 'i\x92ll': 95, '``': 90, "''": 86, 'i-i': 81, 'pheebs': 78, 'he\x92s': 76, 'i\x92ve': 75, 'didn\x92t': 75, "'": 68, 'she\x92s': 59, 'there\x92s': 58, '’': 58, 'let\x92s': 52, 'what\x92s': 49, 'they\x92re': 43, '..': 42, 'doesn\x92t': 40, 'i\x92d': 33, ':': 31, '\x91cause': 31, 'uh-huh': 29, 'no-no': 29, 'we\x92ll': 28, 'wouldn\x92t': 27, 'and-and': 26, 'you\x92ve': 24, 'you\x92ll': 22, 'you-you': 21, 'won\x92t': 21, 'couldn\x92t': 19, 'who\x92s': 19, 'no-no-no': 18, 'wasn\x92t': 18, 'haven\x92t': 17, '\x91em': 16, 'i\x92m-i\x92m': 16, 'i-i-i': 16, 'isn\x92t': 15, 'here\x92s': 14, 'hey-hey': 14, 'uh-oh': 13, 'what-what': 13, 'aren\x92t': 12, 'you\

We also kept track of the *known* words, so lets check these as well:

In [25]:
known_words_count = Counter(known_words)
print('Number of known words',len(known_words_count))
print('Number of known words tokens:', len(known_words))
print('Known words counts')
print(known_words_count)

Number of known words 5535
Number of known words tokens: 45952
Known words counts
Counter({'I': 3424, 'Oh': 1018, 'You': 512, 'Yeah': 510, 'know': 502, 'Okay': 502, 'na': 482, 'No': 471, 'Hey': 441, 'right': 430, 'What': 425, 'Well': 418, "n't": 401, 'gon': 365, 'like': 350, 'go': 347, 'get': 341, 'got': 321, "'m": 299, 'think': 294, 'So': 279, 'really': 262, 'one': 250, 'And': 248, 'mean': 241, 'Ross': 214, 'want': 205, 'All': 201, 'uh': 199, 'God': 195, 'see': 194, "'re": 194, 'Joey': 185, 'tell': 180, 'sorry': 176, 'It': 173, 'okay': 163, 'guys': 162, 'look': 161, 'That': 158, 'going': 158, 'But': 157, 'could': 153, 'Hi': 150, 'good': 147, 'time': 147, 'would': 146, 'back': 141, 'yeah': 140, 'We': 139, 'great': 138, 'come': 137, 'little': 134, 'say': 130, 'love': 130, 'Chandler': 130, 'something': 126, 'Yes': 124, 'Rachel': 119, 'well': 118, 'guy': 118, 'Monica': 116, 'Phoebe': 116, 'Look': 114, 'wan': 112, 'This': 111, 'Why': 111, 'How': 109, 'The': 106, 'make': 106, 'said': 104, '

Just as in the previous notebook, we need to turn the labels into numerical values:

In [26]:
from sklearn import preprocessing
# first we instantiate a label encode
le = preprocessing.LabelEncoder()
# we fee this encoder with the complete list of labels from our data
le.fit(training_labels)
print(list(le.classes_))
training_classes = le.transform(training_labels)
print(list(training_classes[0:20]))

['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']
[4, 4, 4, 4, 6, 4, 4, 4, 4, 4, 2, 4, 6, 4, 6, 5, 6, 2, 4, 4]


The next steps are the same as for the previous notebook, except that we pass the embedding representations of the training data.

In [27]:
# Split data into training and test sets
# from sklearn.cross_validation import train_test_split  # deprecated in 0.18
from sklearn.model_selection import train_test_split

### we again use a aplit of 80% train and 20% test
docs_train, docs_test, y_train, y_test = train_test_split(
    trainDataVecs, # the tf-idf model
    training_classes, # the category values for each utterance represented as numeric values
    test_size = 0.20 # we use 80% for training and 20% for development
    ) 


In [28]:
print(y_test)
print(type(y_test))

[4 4 0 ... 4 5 4]
<class 'numpy.ndarray'>


In [29]:
print(docs_train[0:5])

[[ 0.13769531  0.19140625  0.06982422 ... -0.01904297  0.11083984
   0.07275391]
 [ 0.02927246  0.03244629 -0.01005859 ... -0.08608399  0.05111084
  -0.14296874]
 [ 0.1368103   0.02407837  0.10354614 ... -0.19647217  0.02563477
  -0.03149414]
 [-0.06994629  0.0707194  -0.04036458 ... -0.00495402  0.02033234
  -0.05684407]
 [ 0.11912028  0.10304467  0.0969162  ... -0.01472982 -0.01257324
   0.02475993]]


## 4. Training and applying the model  <a class="anchor" id ="section4"></a> 

In [30]:
from sklearn import svm
from sklearn.calibration import CalibratedClassifierCV

# We choose a Linear model
linear_model = svm.LinearSVC()

# The next function is needed to get some confidence score from this model. This score is derived by 10-folded cross-validation: *cv=10* and using a sigmoid function
svm_linear_clf = CalibratedClassifierCV(linear_model , method='sigmoid', cv=10)

### we train the classifier through the *fit* function and by passing the training vectors and the training labels as paramters:
svm_linear_clf.fit(docs_train, y_train)

CalibratedClassifierCV(base_estimator=LinearSVC(C=1.0, class_weight=None,
                                                dual=True, fit_intercept=True,
                                                intercept_scaling=1,
                                                loss='squared_hinge',
                                                max_iter=1000,
                                                multi_class='ovr', penalty='l2',
                                                random_state=None, tol=0.0001,
                                                verbose=0),
                       cv=10, method='sigmoid')

In [31]:
# Predicting the Test set results, find macro recall
y_pred_svm_linear = svm_linear_clf.predict(docs_test)

If the data is complex, a non-linear SVM may be preferable. A non-linear SVM uses the kernel-trick to separate positive and negative cases when the data is not lineary correlated. We can initialise such a classifier in the same way as done before.

In [ ]:
nonlinear_model = svm.SVC(probability=True)
svm_nonlinear_clf = CalibratedClassifierCV(nonlinear_model,  method='sigmoid', cv=5)
svm_nonlinear_clf.fit(docs_train, y_train)

In [33]:
# Predicting the Test set results, find macro recall
y_pred_svm_nonlinear = svm_nonlinear_clf.predict(docs_test)

In [34]:
# the next function obtain the scores for all the classes for each text
y_pred_svm_nonlinear_proba= svm_nonlinear_clf.predict_proba(docs_test)
print(y_pred_svm_nonlinear_proba)

[[0.05727301 0.0605564  0.01256164 ... 0.48935933 0.02644762 0.15417167]
 [0.10692003 0.03648545 0.0128729  ... 0.51751337 0.07558563 0.02868661]
 [0.20359886 0.02875817 0.02688259 ... 0.54077241 0.08125842 0.03054965]
 ...
 [0.10476192 0.03855785 0.01295518 ... 0.51650307 0.0746161  0.02900226]
 [0.20225783 0.03805203 0.01360721 ... 0.54356266 0.08296208 0.03007048]
 [0.1464218  0.01729576 0.03563543 ... 0.53027047 0.08528737 0.03218758]]


## 5. Generating the test report  <a class="anchor" id ="section5"></a> 

In [35]:
from sklearn.metrics import classification_report


In [36]:
#### this report gives the results for the LINEAR classifier
report = classification_report(y_test,y_pred_svm_linear,digits = 7)
print(le.classes_)
print('SVM LINEAR ----------------------------------------------------------------')
print(report)

['anger' 'disgust' 'fear' 'joy' 'neutral' 'sadness' 'surprise']
SVM LINEAR ----------------------------------------------------------------
              precision    recall  f1-score   support

           0  0.1500000 0.0146341 0.0266667       205
           1  0.0000000 0.0000000 0.0000000        58
           2  0.0000000 0.0000000 0.0000000        58
           3  0.4827586 0.1517615 0.2309278       369
           4  0.5040369 0.9407966 0.6564026       929
           5  0.6666667 0.0431655 0.0810811       139
           6  0.5882353 0.2916667 0.3899721       240

    accuracy                      0.5050050      1998
   macro avg  0.3416711 0.2060035 0.1978643      1998
weighted avg  0.4559466 0.5050050 0.4030734      1998



/Users/piek/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [37]:
# this report gives the results for the nonlinear classifier
report = classification_report(y_test,y_pred_svm_nonlinear,digits = 7)
print(le.classes_)
print('SVM NONLINEAR ----------------------------------------------------------------')
print(report)

['anger' 'disgust' 'fear' 'joy' 'neutral' 'sadness' 'surprise']
SVM NONLINEAR ----------------------------------------------------------------
              precision    recall  f1-score   support

           0  0.3333333 0.0097561 0.0189573       205
           1  0.0000000 0.0000000 0.0000000        58
           2  0.0000000 0.0000000 0.0000000        58
           3  0.5955056 0.1436314 0.2314410       369
           4  0.4991559 0.9547901 0.6555802       929
           5  0.6666667 0.0431655 0.0810811       139
           6  0.5775862 0.2791667 0.3764045       240

    accuracy                      0.5080080      1998
   macro avg  0.3817497 0.2043585 0.1947806      1998
weighted avg  0.4920311 0.5080080 0.4003650      1998



Remember the results from the notebook where we trained a NaiveBayes and SVM classifiers with one-hot-encodings of the words? Take some time to compare the results and think about the differences.

## 6. Applying the model to new data  <a class="anchor" id ="section6"></a> 

We would like to apply the embedding based model to our own data but this works a bit different as we cannot simply use the 'transform' function to represent the utterances using the one-hot vector representation of the training vocabulary.

What we need to do is to create an embedding representation using the same function we used above and assume that our classifier finds sufficient similarity in the embeddings of our data with the correct training data.

We use the same set of utterances.

In [39]:
# some utterances
some_chat = ['That is sweet of you', 
               'You are so funny', 
               'Are you a man or a woman?', 
               'Chatbots make me sad and feel lonely.', 
               'Your are stupid and boring.', 
               'Two thumbs up', 
               'I fell asleep halfway through this conversation', 
               'Wow, I am really amazed.', 
               'You are amazing.']


len(some_chat)

9

Next, we define the list of labels that go with our chat.

In [40]:
some_chat_emotions = ['joy', 'joy', 'neutral', 'sadness', 'anger', 'joy', 'anger', 'surprise', 'joy']

We  use the LabelEncoder *le* to convert this list into a numpy array with digits:

In [41]:
print('labels',le.classes_)
some_chat_labels = le.transform(some_chat_emotions)
print(some_chat_labels)

labels ['anger' 'disgust' 'fear' 'joy' 'neutral' 'sadness' 'surprise']
[3 3 4 5 0 3 0 6 3]


In [44]:
some_chat_tokens = []
for utterance in some_chat:
    some_chat_tokens.append(nltk.tokenize.word_tokenize(utterance))

some_chat_embedding_vectors = getAvgFeatureVecs(some_chat_tokens, stop_words, word_embedding_model, index2word_set, num_features)
#### Due to the averaging, there could be infinitive values or NaN values. The next numpy function turns these value to "0" scores
some_chat_embedding_vectors = np.nan_to_num(some_chat_embedding_vectors)  

Review 0 of 9


In [46]:
# have classifier make a prediction
#pred = svm_linear_clf.predict(ourDataVecs)
from sklearn.calibration import CalibratedClassifierCV

some_chat_pred = svm_linear_clf.predict(some_chat_embedding_vectors)
print('System predictions', some_chat_pred)
print('Gold labels', some_chat_labels)
for review, predicted_label in zip(some_chat, some_chat_pred):
    
    print('%s => %s' % (review, 
                        le.classes_[predicted_label]))




System predictions [3 3 4 4 0 3 4 6 3]
Gold labels [3 3 4 5 0 3 0 6 3]
That is sweet of you => joy
You are so funny => joy
Are you a man or a woman? => neutral
Chatbots make me sad and feel lonely. => neutral
Your are stupid and boring. => anger
Two thumbs up => joy
I fell asleep halfway through this conversation => neutral
Wow, I am really amazed. => surprise
You are amazing. => joy


In [47]:
some_chat_pred_probabilities = svm_linear_clf.predict_proba(some_chat_embedding_vectors)
print(some_chat_pred_probabilities)

[[0.04761473 0.01158879 0.0162128  0.47619004 0.34488579 0.05194794
  0.05155991]
 [0.14686937 0.01795713 0.03158916 0.36852899 0.15075793 0.03478174
  0.24951567]
 [0.20240196 0.03007896 0.02393157 0.02459954 0.40257576 0.04715669
  0.26925552]
 [0.16862088 0.02253751 0.04177757 0.16615773 0.33473105 0.24561419
  0.02056106]
 [0.2932923  0.0791925  0.02004016 0.15719427 0.26723984 0.06413371
  0.11890721]
 [0.07028081 0.03052093 0.03224515 0.49416629 0.3127675  0.00351845
  0.05650087]
 [0.18037703 0.02289737 0.01605887 0.04442399 0.53031717 0.04648082
  0.15944475]
 [0.046185   0.03339277 0.02868321 0.34612863 0.08314988 0.03093737
  0.43152313]
 [0.03205453 0.01786641 0.01521225 0.65378362 0.03539541 0.02985222
  0.21583556]]


Using *Pandas* we can nicely visualise the results. 

In [48]:
some_chat_pred_labels = []
for predicted_label in some_chat_pred:
    some_chat_pred_labels.append(le.classes_[predicted_label])

some_chat_gold_labels = []
for gold_label in some_chat_labels:
    some_chat_gold_labels.append(le.classes_[gold_label])


result_frame = pd.DataFrame(some_chat_pred_probabilities*100, columns=le.classes_)
result_frame['Chat']=some_chat
result_frame['Predication']=some_chat_pred_labels
result_frame['Gold']=some_chat_gold_labels

result_frame

,anger,disgust,fear,joy,neutral,sadness,surprise,Chat,Predication,Gold
0,4.761473,1.158879,1.621280,47.619004,34.488579,5.194794,5.155991,That is sweet of you,joy,joy
1,14.686937,1.795713,3.158916,36.852899,15.075793,3.478174,24.951567,You are so funny,joy,joy
2,20.240196,3.007896,2.393157,2.459954,40.257576,4.715669,26.925552,Are you a man or a woman?,neutral,neutral
3,16.862088,2.253751,4.177757,16.615773,33.473105,24.561419,2.056106,Chatbots make me sad and feel lonely.,neutral,sadness
4,29.329230,7.919250,2.004016,15.719427,26.723984,6.413371,11.890721,Your are stupid and boring.,anger,anger
5,7.028081,3.052093,3.224515,49.416629,31.276750,0.351845,5.650087,Two thumbs up,joy,joy
6,18.037703,2.289737,1.605887,4.442399,53.031717,4.648082,15.944475,I fell asleep halfway through this conversation,neutral,anger
7,4.618500,3.339277,2.868321,34.612863,8.314988,3.093737,43.152313,"Wow, I am really amazed.",surprise,surprise
8,3.205453,1.786641,1.521225,65.378362,3.539541,2.985222,21.583556,You are amazing.,joy,joy


In [50]:
report = classification_report(some_chat_labels,some_chat_pred,digits = 7)
print(le.classes_)
print('SVM LINEAR ----------------------------------------------------------------')
print(report)

['anger' 'disgust' 'fear' 'joy' 'neutral' 'sadness' 'surprise']
SVM LINEAR ----------------------------------------------------------------
              precision    recall  f1-score   support

           0  1.0000000 0.5000000 0.6666667         2
           3  1.0000000 1.0000000 1.0000000         4
           4  0.3333333 1.0000000 0.5000000         1
           5  0.0000000 0.0000000 0.0000000         1
           6  1.0000000 1.0000000 1.0000000         1

    accuracy                      0.7777778         9
   macro avg  0.6666667 0.7000000 0.6333333         9
weighted avg  0.8148148 0.7777778 0.7592593         9



/Users/piek/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 7. Saving the classifier to disk

Just as with the previous notebook, you can save the emotion classification model to disk and load the model some other time. Note that you need to load the same word2vec model as well to represent any text input with vector representations that are compatible.

In [51]:
import pickle

# save the classifier to disk
filename_classifier = 'svm_nonlinear_clf_embeddings.sav'
pickle.dump(svm_nonlinear_clf, open(filename_classifier, 'wb'))
 
# some time later...
 
# load the classifier and the vectorizer from disk
loaded_classifier = pickle.load(open(filename_classifier, 'rb'))


# End of this notebook